In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io, filters
import numpy as np

from tqdm.notebook import tqdm

from PIL import Image
import numpy as np

import cv2
import copy
import time

from mpire import WorkerPool

from PIL import Image
from matplotlib import cm
from tqdm.notebook import tqdm


from combra import stats as cstats
from combra import approx as capprox
from combra import image as cimage
from combra import contours as ccontours

from numba import njit
import numpy as np
import cv2
import os

from combra.tests import test_fractal_dimensions
from combra.contours import contour_to_binary_mask, scale_contour, draw_contours

import json
from pathlib import Path

import pandas as pd

import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = img[400:-400,1300:-1300]
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)

    cnts, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    return binary, cnts


# --- Group images by number ranges (1-500, 501-1000, ...) ---
from pathlib import Path
import re


def _extract_image_number(p):
    stem = Path(p).stem
    m = re.search(r"(\d+)", stem)
    return int(m.group(1)) if m else None


images_dir = Path("data/autumn/images")
all_image_paths = [p for p in images_dir.glob("*.JPG") if re.fullmatch(r"\d+", p.stem)]
all_image_paths = sorted(
    all_image_paths,
    key=lambda p: _extract_image_number(p) or 0,
)

# If you also have .jpg/.jpeg, uncomment:
# all_image_paths += list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.jpeg"))

group_size = 250
_groups = {}

for p in all_image_paths:
    n = _extract_image_number(p)
    if n is None:
        continue
    g_start = ((n - 1) // group_size) * group_size + 1
    g_end = g_start + group_size - 1
    label = f"{g_start:04d}-{g_end:04d}"
    _groups.setdefault(label, []).append(str(p))

# Sorted list of (label, [image_path, ...])
image_groups = sorted(
    _groups.items(),
    key=lambda kv: int(kv[0].split("-")[0]),
)

group_labels = [label for label, _ in image_groups]

print(f"Found {len(all_image_paths)} images, grouped into {len(image_groups)} groups")
if image_groups:
    print("First 5 groups:", [(k, len(v)) for k, v in image_groups[:20]])



# Fractal dimention

In [ ]:
sizes = 2 ** np.arange(1, 10)
test_fractal_dimensions(sizes)

In [ ]:
n_jobs=20


n_images_list = []
n_contours_list = []
n_contours_fd_list = []

# Stream results directly to parquet to keep memory bounded
out_path = Path("poliamid_group_250.parquet")

schema = pa.schema(
    [
        ("group_label", pa.string()),
        ("n_images", pa.int32()),
        ("n_contours_total", pa.int32()),
        ("n_contours_fd", pa.int32()),
        ("fd_list", pa.list_(pa.float64())),
        ("len_nodes_list", pa.list_(pa.int32())),
        ("len_pixels_list", pa.list_(pa.float64())),
        ("area_list", pa.list_(pa.float64())),
        ("n_jobs", pa.int32()),
    ]
)

writer = pq.ParquetWriter(out_path, schema, compression="zstd")


def _metrics_for_image(image_path):
    # Compute metrics inside the worker to avoid returning huge contour objects
    _, contours = preprocess_image(image_path)

    n_total = len(contours)

    fd_list = []
    ln_list = []
    lp_list = []
    ar_list = []

    for c in contours:
        fd = cimage.contour_fractal_dimension(c)
        if fd is None:
            continue
        fd_list.append(float(fd))
        ln_list.append(int(len(c)))
        lp_list.append(float(cv2.arcLength(c, closed=True)))
        ar_list.append(float(cv2.contourArea(c)))

    return n_total, fd_list, ln_list, lp_list, ar_list


with WorkerPool(n_jobs=n_jobs, use_dill=False) as pool:
    for idx, (group_label, group_paths) in tqdm(enumerate(image_groups)):

        total_contours = 0
        fd_list = []
        ln_list = []
        lp_list = []
        ar_list = []

        # Process per-image in parallel and stream results back (keeps memory bounded)
        for n_total, fds, lns, lps, ars in pool.imap_unordered(
            _metrics_for_image,
            group_paths,
            progress_bar=True,
            chunk_size=1,
        ):
            total_contours += int(n_total)
            fd_list.extend(fds)
            ln_list.extend(lns)
            lp_list.extend(lps)
            ar_list.extend(ars)

        # Keep some diagnostics
        n_images_list.append(len(group_paths))
        n_contours_list.append(total_contours)
        n_contours_fd_list.append(len(fd_list))

        # Write one row per group (keeps RAM bounded)
        row = pa.table(
            {
                "group_label": pa.array([group_label], type=pa.string()),
                "n_images": pa.array([len(group_paths)], type=pa.int32()),
                "n_contours_total": pa.array([total_contours], type=pa.int32()),
                "n_contours_fd": pa.array([len(fd_list)], type=pa.int32()),
                "fd_list": pa.array([fd_list], type=pa.list_(pa.float64())),
                "len_nodes_list": pa.array([ln_list], type=pa.list_(pa.int32())),
                "len_pixels_list": pa.array([lp_list], type=pa.list_(pa.float64())),
                "area_list": pa.array([ar_list], type=pa.list_(pa.float64())),
                "n_jobs": pa.array([n_jobs], type=pa.int32()),
            },
            schema=schema,
        )
        writer.write_table(row)

# Close parquet writer
writer.close()
print("Saved:", out_path.resolve())


In [ ]:
img_path = image_groups[0][1][100]


_, cnts = preprocess_image(img_path)

# 56 
# 11
contour = cnts[11]

binary = contour_to_binary_mask(contour, thickness=1)

print(len(contour))
plt.imshow(binary, cmap='gray')
plt.savefig('binary.jpg', bbox_inches='tight')
plt.show()

sizes, scale = cimage.valid_box_sizes_from_shape(binary.shape)

cnt_big = ccontours.scale_contour(contour, scale)

binary_big = ccontours.contour_to_binary_mask(cnt_big, thickness=1)

plt.imshow(binary_big, cmap='gray')
plt.savefig('binary_big.jpg', bbox_inches='tight')
plt.show()

D = cimage.image_fractal_dimension(binary_big, sizes)

print(D)


In [ ]:
# Use Parquet generated by the fractal-dimension extraction cell
in_path = Path("poliamid_group_250.parquet")

table = pq.read_table(in_path)
data = table.to_pydict()


In [ ]:

n = len(data["group_label"])
groups = [{k: data[k][i] for k in data.keys()} for i in range(n)]
group_labels = [g["group_label"] for g in groups]

# bin size for stats_preprocess
for step in tqdm([0.01, 0.05, 0.1, 0.2, 0.3]):

    fig, axes = plt.subplots(1, 2, figsize=(20, 7))

    # Colors per group (will cycle if many groups)
    colors = plt.cm.tab20(np.linspace(0, 1, max(2, min(20, len(groups)))))

    a_list = []

    for idx, g in enumerate(groups):
        fd_list = g.get("fd_list", [])
        if fd_list is None:
            fd_list = []
        else:
            fd_list = list(fd_list)

        color = colors[idx % len(colors)]

        if not fd_list:
            a_list.append(np.nan)
            continue

        x_orig, y_orig = cstats.stats_preprocess(fd_list, step)

        # Exponential distribution only
        (x_fit, y_fit), a, amp_exp = capprox.exponential_approx(
            x_orig, y_orig, a=1, amp=1, x_lim=[1, 2], N=20
        )
        a_list.append(a)

        label = (
            # f"{g['group_label']} | imgs={g.get('n_images', 0)} | cnts={g.get('n_contours', 0)} | a={a:.4f}"
            f"{g['group_label']} | a={a:.4f}"
        )

        axes[0].plot(x_fit, y_fit, '--', linewidth=2, color=color)
        axes[0].plot(x_orig, y_orig, '-o', color=color, label=label)

    axes[0].set_title(f'exponential fit, step={step}', fontsize=15)
    axes[0].set_xlim(1, 2)
    axes[0].set_ylim(1e-8, 1)
    axes[0].set_yscale('log')
    axes[0].set_ylabel('p(x)', fontsize=15)
    axes[0].set_xlabel('fractal dimension', fontsize=15)
    axes[0].legend(loc='upper right', fontsize=10)

    # Diagnostics: parameter a per group
    axes[1].plot(range(len(groups)), a_list, '-o', label='a')
    axes[1].set_title(f'exponential parameter a, step={step}', fontsize=15)
    axes[1].set_xticks(range(len(groups)))
    axes[1].set_xticklabels(group_labels, rotation=45, fontsize=12, ha='right')
    axes[1].legend(loc='upper right', fontsize=12)

    plt.savefig(f'fractal_dimention_step={step}.jpg', bbox_inches='tight')

    plt.tight_layout()
    plt.show()
